In [1]:
import numpy as np
import sklearn as sk  # namespace conflict with pyspark
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from helper import *
import tensorflow as tf

pd.options.display.max_columns = None
import findspark
import pyspark
from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from pyspark.sql.functions import col

from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator

# from keras.layers import Dense
# from keras.models import Model, Sequential
# from keras import initializers, optimizers
# from keras import backend as K

# Stuff that needs to be done

- Find another framework.
    - Sklearn
    - Spark
    - Tensorflow/R
- Algorithms
    - Linear Regression
    - Logistic Regression
    - Maybe SVM /Random forest
- Find two metrics 
    - RMSE 
    - Some other
- Find another dataset 
    - Sum data with noise 
    -  Maybe housing data set
    

# Scikitlearn


## Import DataSet

In [16]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAD6JGlvG5XADIjg9SCojvpya/House%20Sales%20in%20King%20County%2C%20USA?dl=1"
all_urls = [sumdata_url, housing_price_url]

In [17]:
get_data(all_urls) # retrieves the data if there is no data folder

In [51]:
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
housing_price_path = "data/kc_house_data.csv" # has more than 30 features
# need one more
# what a brilliant idea to name files with space

In [52]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

## Load datasets sum

In [53]:
sumdata_noise= pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise.head(n=3)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless but please still use it),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,62485,58472,84200,86181,75529,136939,150633,230058,246491,257336,1352179,Very Large Number
1,2,75559,119137,146760,139674,19582,177083,217746,321110,434444,516798,1976446,Very Large Number
2,3,26568,30742,28693,36891,24065,60797,79331,84997,116275,129739,558391,Very Large Number


## Preprocess sum dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract rest of the columns as explananatory variables, except Instance and Target, Target Class
- Ensure all dataframe has been converted to numpy array


In [54]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

sumdata_noise.drop('Instance', axis = 1)

# Extract 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise.loc[:, ['Noisy Target']]
# Extract 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise.loc[:, ['Noisy Target Class']] 
 
 
 
sumdata_noise_classif_Y ['Noisy Target Class'] = le.fit_transform( sumdata_noise_classif_Y ['Noisy Target Class'].astype('str')) 
 
# Extract rest columns as explananatory variables
sumdata_noise_X =sumdata_noise.drop(['Instance','Noisy Target Class','Noisy Target'
                                    ], axis = 1) 
# our SPARK NEEDS target column to be named "label"
sumdata_noise_reg_Y.rename(columns={'Noisy Target': 'label'}, inplace=True)
sumdata_noise_classif_Y.rename(columns={'Noisy Target Class': 'label'}, inplace=True)


## Load datasets house price

In [23]:
housing_price = pd.read_csv(housing_price_path, delimiter=",")
housing_price.head(n=10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


## Preprocess house price dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest of the columns as explananatory variables

- Ensure all dataframe has been converted to numpy array


In [24]:
# Remove 'Instance' as it simply represents the row number

# Extract 'Nosiy Target' as regression target
housing_price_reg_Y = housing_price.loc[:, ['price']]

# get all the numerical data that is needed
explanatory_numeric = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'condition', 'sqft_above', 'yr_built',
                      'sqft_living15', 'sqft_lot15']

housing_price[explanatory_numeric[0]]
filtered_table = housing_price[explanatory_numeric[0]]
filtered_table

for c in explanatory_numeric[1:]:
    filtered_table = pd.concat([filtered_table, housing_price[c]], axis=1)
filtered_table

# get explanatory variable
housing_price_reg_X = filtered_table[:].values
# housing_price_classif_X = filtered_table[:].values

# get the regression target and classification target
# I am using sales price as both regression and classification target
# sale_price_classi = SalePrice<Mean, SalePrice>=Mean
housing_price_reg_Y = housing_price['price'].values

# housing_price_classif_Y = np.zeros(housing_price_reg_Y.shape)
# house_price_mean = housing_price_reg_Y.mean()

# for i, price in enumerate(housing_price_reg_Y):
#     if price >= house_price_mean:
#         housing_price_classi_Y[i] = 1
#     else:
#         housing_price_classi_Y[i] = 0

# # Apply Feature Scaling to the numeric variables and regression target 
# scX = StandardScaler()
# scY = StandardScaler()
# housing_price_classif_X = scX.fit_transform(housing_price_classif_X)
# housing_price_classif_Y = scY.fit_transform(housing_price_classif_Y.reshape(-1, 1))





In [25]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


In [55]:
def spark_random_forest( X_train, X_test, y_train, y_test):
    """
        learning rate: cant configure it
        epochs: 10 (maxiter)
        batch size 1 
        objective function is ordinary least squared error
        root mean squared error for evaluating testdata
    """
    from pyspark.ml import Pipeline
    from pyspark.ml.regression import RandomForestRegressor
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    train_data = pd.concat([X_train,y_train], axis =1)
    test_data  = pd.concat([X_test,y_test], axis =1)
    
 
    train_data_df = sqlContext.createDataFrame(train_data)
    test_data_df = sqlContext.createDataFrame(test_data) 
   
     
    cols = train_data.columns[:-1].values

    assembler = (VectorAssembler()
        .setInputCols(cols) # everything excluding target
        .setOutputCol("features"))
 
    rfr = RandomForestRegressor

    pipeline = Pipeline(stages=[assembler, rfr])
    model = pipeline.fit(train_data_df)
    
    
    predictions = model.transform(test_data_df) 
    
    #predictions.select("predictions", "label").toPandas()  
    evaluator = RegressionEvaluator(metricName="rmse")
    RMSE = evaluator.evaluate(predictions)
    evaluator = RegressionEvaluator(metricName="mae")
    MAE = evaluator.evaluate(predictions)
  
    return RMSE,MAE
    

In [56]:
model_test_split( sumdata_noise_X.head(n=10), sumdata_noise_reg_Y.head(n=10), "Sum data with noise", spark_random_forest)

TypeError: Cannot recognize a pipeline stage of type <class 'type'>.

In [39]:
def sklearn_random_forest( X_train, X_test, y_train, y_test): 
    """
        n_estimators 100
        criterion mae
        max_features size auto
        objective function is ordinary least squared error
        root mean squared error for evaluating testdata
       
    """
    from sklearn.ensemble import RandomForestRegressor
    rfr = RandomForestRegressor()
    model = rfr.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae =  sk.metrics.mean_absolute_error(y_test, preds)
    rmse = sk.metrics.mean_squared_error(y_test, preds)**0.5
    return (rmse,mae)

### 70/30 split 

In [28]:
def model_test_split( data_x, data_y, dataset_name, algorithm): 
    algorithm_n = algorithm.__name__

    X = data_x
    y = data_y

    kf = KFold(n_splits=10)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)


        # accumulating errors for each fold
    error = algorithm( X_train, X_test, y_train, y_test) 

    print ("Algorithm: {}  Data set: {} Error: {}".format(algorithm_n, dataset_name, error))
        

In [29]:
model_test_split( sumdata_reg_X, sumdata_reg_Y.ravel(), "Sum data with noise", sklearn_random_forest)

/home/jibin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


KeyboardInterrupt: 

In [41]:
model_test_split( sumdata_reg_X.head(n=10), sumdata_reg_Y.head(n=10), "Sum data with noise", spark_random_forest)

TypeError: Invalid param value given for param "inputCols". Could not convert Index(['Feature 1', 'Feature 2', 'Feature 3', 'Feature 4',
       'Feature 5 (meaningless but please still use it)', 'Feature 6',
       'Feature 7', 'Feature 8', 'Feature 9', 'Feature 10'],
      dtype='object') to list of strings

In [32]:
sumdata_reg_Y

,Target
0,1369233
1,2098516
2,566579
3,591657
4,1143179
5,842812
6,336066
7,812530
8,868923
9,617838
